# 1.0 Setup

Ensure that your Jupyter environment is setup correctly and import all of the data science libraries that we will need.  If some modules are missing, we will attempt to install the library but it is usually a better practice to install it in your environment directly.

Also, if the data is missing, we will attempt to download it (from github) and put it in the "/data" subdirectory of your current working directory.

In [ ]:
# Import required modules
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib as mplot
%matplotlib inline
import IPython
from IPython.core.display import HTML
from IPython.core.debugger import set_trace
from distutils.version import StrictVersion
print("numpy version:  %s" % np.__version__)
print("pandas version:  %s" % pd.__version__)
print("matplotlib version:  %s" % mplot.__version__)
print("IPython version:  %s" % IPython.__version__)
print("seaborn version:  %s" % sns.__version__)

if StrictVersion(np.__version__) >= StrictVersion('1.13.0') and \
   StrictVersion(pd.__version__) >= StrictVersion('0.20.0') and \
   StrictVersion(mplot.__version__) >= StrictVersion('2.0.0') and \
   StrictVersion(IPython.__version__) >= StrictVersion('5.5.0') and \
   StrictVersion(sns.__version__) >= StrictVersion('0.7.0'):
    print('\nCongratulations, your environment is setup correctly!')
else:
    print('\nEnvironment is NOT setup correctly!')


In [ ]:
# Try to install the Excel reader library

try:
    import xlrd
    print('The Excel library is installed.')
except ImportError:
    print('Installing the Excel library')
    !pip install xlrd
    import xlrd

### 1.1 Check data directory

See if the data exists.  If not, try to download it from github.

In [ ]:
# Find the data directory and download data if it is missing

import os, shutil
cwd = os.getcwd()
datadir = cwd + '/data'

print('Data directory is: {}'.format(datadir))

In [ ]:
# See if the data exists.  If not, try to download it from github.
if not os.path.exists(datadir+'/patients.csv'):
    print("Data directory doesn't exist!")
    # Checkout the data from Github
    !git clone https://github.com/stevejohnson2001/dswg
    shutil.move('dswg/data','.')  # Move the checked-out files into the /data directory
    shutil.rmtree('dswg')  # Remove the version control (git) information
print('Data directory contains:\n',os.listdir(datadir))


# 2.0 Read in the data

Now that we have all the libraries installed and the data is available, lets try to read it into Pandas DataFrames.  

The first thing we will do is define a Data Dictionary (dd) that describes our expectations for the data.  It includes data types for the columns as well is information about whether the column is required or optional.  The data is read into a dictionary of Dataframes (data) and also assigned to  variables (patients, encounters, etc) for convenience.

We will convert dates and other fields to the proper format when later when we do data preparation.

The dataset is synthetic data generated by the Synthea project (https://github.com/synthetichealth/synthea).  Synthea creates realistic (but not real) EHR-like data that we can use for demonstrating the techniques of data science.

In [ ]:
# Read in the data
dd = {}

dd['patients'] = {'pat_id':     {'type': np.str, 'required':True},  
                  'birth_date': {'type': np.datetime64, 'format': '%Y-%m-%d', 'required':True},
                  'death_date': {'type': np.datetime64, 'format': '%Y-%m-%d' }, 
                  'ssn':        {'type': np.str},
                  'drivers':    {'type': np.str},
                  'passport':   {'type': np.str},
                  'prefix':     {'type': np.str},
                  'first':      {'type': np.str, 'required':True},
                  'last':       {'type': np.str, 'required':True},
                  'suffix':     {'type': np.str},
                  'maiden':     {'type': np.str},
                  'marital':    {'type': np.str},
                  'race':       {'type': np.str},
                  'ethnicity':  {'type': np.str},
                  'gender':     {'type': np.str, 'required':True},
                  'birthplace': {'type': np.str},
                  'address':    {'type': np.str, 'required':True},
                  'prior_opioid_abuse_diag': {'type': np.int}
                  }
dd['encounters'] = {'enc_id':                 {'type': np.str, 'required':True}, 
                    'enc_date':               {'type': np.datetime64, 'format': '%Y-%m-%d', 'required':True},
                    'enc_pat_id':             {'type': np.str, 'required':True},
                    'enc_code':               {'type': np.str, 'required':True},
                    'enc_description':        {'type': np.str, 'required':True},
                    'enc_reason_code':        {'type': np.str},
                    'enc_reason_description': {'type': np.str}
                   }
dd['observations'] = {'obs_date':        {'type': np.datetime64, 'format': '%Y-%m-%d', 'required':True},
                      'obs_pat_id':      {'type': np.str, 'required':True},
                      'obs_enc_id':      {'type': np.str, 'required':True},
                      'obs_code':        {'type': np.str, 'required':True},
                      'obs_description': {'type': np.str, 'required':True},
                      'obs_value':       {'type': np.str},
                      'obs_units':       {'type': np.str}
                     }
dd['medications'] = {'med_start_date':         {'type': np.datetime64, 'format': '%Y-%m-%d', 'required':True},
                     'med_stop_date':          {'type': np.datetime64, 'format': '%Y-%m-%d', 'required':False},
                     'med_pat_id':             {'type': np.str, 'required':True},
                     'med_enc_id':             {'type': np.str, 'required':True},
                     'med_code':               {'type': np.str, 'required':True},
                     'med_description':        {'type': np.str, 'required':True},
                     'med_reason_code':        {'type': np.str},
                     'med_reason_description': {'type': np.str},
                     'med_days_supply':        {'type': np.int}
                     }
dd['conditions'] =  {'cond_start_date':         {'type': np.datetime64, 'format': '%Y-%m-%d', 'required':True},
                     'cond_stop_date':          {'type': np.datetime64, 'format': '%Y-%m-%d', 'required':False},
                     'cond_pat_id':             {'type': np.str, 'required':True},
                     'cond_enc_id':             {'type': np.str, 'required':True},
                     'cond_code':               {'type': np.str, 'required':True},
                     'cond_description':        {'type': np.str, 'required':True}
                     }


# Display the data dictionary
# Use HTML to make it look a little nicer

for name, tbl_dd in dd.items():
    display(HTML('<h2>{}</h2>'.format(name)))
    display(pd.DataFrame(tbl_dd).fillna('').T)

In [ ]:
# Loop through each of the file defintions in our data dictionary
data = {}
for f in dd:
    m = dd[f]
    col_names = list(m.keys())
    data[f] = pd.read_csv(datadir + '/{}.csv'.format(f), dtype=str, index_col=False, header=0, \
                          names=col_names, keep_default_na=False)

# Assign data to local variables for convenience
patients = data['patients']
encounters = data['encounters']
observations = data['observations']
medications = data['medications']
conditions = data['conditions']

### 2.1 Load the list of opioid medications

It will be important to know which of the medications that are prescribed are considered opioids.  The UMLS VSAC maintains value set lists of which medications are considered opioids.  You can download the current list by going to https://vsac.nlm.nih.gov/ and searching for opioid value sets.  We have downloaded the list called "All prescribable opioids used for pain control including Inactive Medications" (oid 1.3.6.1.4.1.6997.4.1.2.234.999.3.2) into the data directory.  We will read the Excel file and pull out the codes from the second sheet in the file.

In [ ]:
# Get Opioid code list from VSAC
# oid 1.3.6.1.4.1.6997.4.1.2.234.999.3.2
xl = pd.ExcelFile(datadir + '/AllPrescribableOpioidsUsedForPainControlIncludingInactiveMedications.xlsx')
df = xl.parse("Code List", skiprows=12)
display(df.head(10))
opioids_rxnorm = list(df['Code'].astype(np.str))  # Make sure the codes are treated as strings


# 3.0 Exploratory Data Analysis - Part 1

Start by looking at the data to see what types of values are in each variables, the relationships and get a feel for the data.

### 3.1 Start by displaying the data as DataFrames

Displaying the first few rows of the data is a good way to look for obvious issues before working with the data in more detail.  


In [ ]:
# Loop through each of the file defintions in our data dictionary
for f in dd:
    m = dd[f]
    display(HTML('<h3>{}, {} records</h3>'.format(f,len(data[f]))))
    display(data[f].head(5))

In [ ]:
# You can also display the last few rows using the .tail() function.
patients.tail(5)

### 3.2 Categorical Variables

In [ ]:
# Look at the categorical variables

sns.countplot(x='race', data=patients)
plt.title('Race')
plt.show()

sns.countplot(x='obs_code',data=observations)
plt.title('Observation Codes')
plt.show()

In [ ]:
# The observations are hard to read, lets try a bar plot
c = observations['obs_description'].value_counts(ascending=True)
fig = plt.figure(figsize=(8,24))
c.plot(kind="barh")

### 3.3 Continuous Variables

In [ ]:
# For continuous variables, we can graph the distribution

w = observations[observations['obs_code']=='29463-7']   # Find all of the "weight" observations
weights = w['obs_value'].astype(np.float)
mean = np.mean(weights)
print('Average weight: ',mean)

# Plot the distribution
sns.distplot(weights)
plt.xlabel("WEIGHT (kg)")
plt.show()   # If you don't explicitly "show" the plot, Jupyter will automatically show the last plot

In [ ]:
# As an Exercise, have participants graph the Height of patients



# 4.0 Data Preparation

Now that we know a little about our data, we can begin to prepare it for analysis.  We need to:
1. Find data that is not formatted correctly
2. Deal with missing data

In all of these cases, we will have to decide what to do with the bad data.  We can:
1. Delete the data
2. Impute a reasonable value for the missing data

### 4.1 Data Quality Checks

Start by running a few data quality checks against all of the data.  For this workshop, we will only look at missing data or incorrectly formatted data but with a real data science project you would also want to check that the relationships between all of the data elements makes sense.  For example, you'd ensure that the `birth_date` is less than today.

The functions below attempt to convert the data (which we read in as simple strings) to the data format that we defined for each variable.  That's how we will tell if it is in the correct format.

In [ ]:
# Find data that is not formatted correctly

def parse_date(dt,fmt):
    if type(dt) == str and (dt == ''):
        return dt
    try:
        return pd.to_datetime(dt,format=fmt)
    except:
        return np.datetime64('NaT')

def parse_int(num):
    if type(num) == str and (num == ''):
        return num
    try:
        return int(num)
    except:
        return np.nan

# Loop through our Data Dictionary
for name, tbl_dd in dd.items():
    display(HTML('<h2>{}</h2>'.format(name)))
    d = data[name]
    for field_name, field in tbl_dd.items():
        col = d[field_name]
        field['DQ'] = {}
        field['DQ']['missing'] = len(np.where(col == '')[0])

        if field['type'] == np.datetime64:
            if 'format' in field:
                fmt = field['format']
            else:
                fmt = '%Y-%m-%d'   # Default date format if not specified
                
            d[field_name] = col.apply(lambda x: parse_date(x,fmt))
            field['DQ']['format_errors'] = col.isnull().sum()
        elif field['type'] == np.int:
            d[field_name] = col.apply(lambda x: parse_int(x))
            field['DQ']['format_errors'] = col.isnull().sum()
        elif field['type'] == np.str:
            pass # Everything is valid syntax
            field['DQ']['format_errors'] = 0
            
    # Show the Data Quality information
    display(pd.DataFrame(dd[name]))
    


### 4.2 Drop unwanted data

In [ ]:
# Let's remove some of the data that we won't need to use for the workshop
# It will make some of the screens easier to read
# Put it in a try block in case we've already dropped the columns
try:
    patients.drop(['maiden','passport','drivers','prefix','suffix','ssn','first','last'],axis=1,inplace=True)
except:
    pass
display(patients.head(1))

### 4.3 Print the Data Dictionary

In [ ]:
# Print Data Dictionary
# TODO: Move this
for name, tbl_dd in dd.items():
    display(HTML('<h2>{}</h2>'.format(name)))
    list(dd[name].keys())
    d = pd.DataFrame(dd[name]).fillna('')
    d = d[list(dd[name].keys())]
    d = d.replace(np.str,'str')
    d = d.replace(np.datetime64,'datetime')
    d = d.replace(np.int,'int')
    d = d.loc[['type','format','required'],:]
    if name == 'patients':
        d = d.drop(['maiden','passport','drivers','prefix','suffix','ssn','first','last'],axis=1)
    for fname, field in d.items():
        if fname.endswith('date'):
            field['type'] = 'datetime'
    display(d)

### 4.3 Remove rows with missing data

For this workshop, we will only deal with missing data by dropping it.  If a row has missing or bad formatted data and the field is required, we will drop the entire row.

An alternative is to try to fix the data by imputing a reasonable value for it, such as the column .mean().

In [ ]:
# The .isnull() functions are used to find bad data
# The .any() function returns the columns that contain any True values

display(patients.isnull().head(5))
display(patients.isnull().any())

# Let's get a list of all of the columns with some missing data 
missing_cols=patients.columns[patients.isnull().any()]
print(missing_cols)

# We can see how many cells have missing data for each column
patients[missing_cols].isnull().sum()


In [ ]:
# Drop missing or incorrectly formated data

# TODO: Do for all Data

# Get the row numbers (index) of each of the rows with missing data
missing = patients[patients.isnull().any(axis=1)].index
print("Missing = ",missing)
print('Before patients shape = ',patients.shape)
patients.drop(missing,inplace=True)

# Make sure the rows are gone
missing = patients[patients.isnull().any(axis=1)].index
print("Missing = ",missing)
print('After patients shape = ',patients.shape)




## 4.1 Transform the Data

Use the power of Pandas Dataframes to transform the data.  Add new columns as calculations from existing columns, join the data together and get it into the format you need for analysis.



In [ ]:
# Create the working dataframe
# TODO cleanup

df = patients
df['age'] = round((pd.Timestamp.today() - pd.to_datetime(patients['birth_date'])).dt.days/365)
df['adult'] = np.where(df['age'] >= 18, 1, 0)

# Determine which patients have ever overdosed
# Use a set to eliminate duplicates
patients_that_overdosed = set(encounters[encounters['enc_reason_code']=='55680006']['enc_pat_id'])  # Overdose
df['overdose'] = np.where(df['pat_id'].isin(patients_that_overdosed), 1, 0)

# Determine which patients have died from an overdose
# Uses binary indexing
patients_overdose_deaths = set(observations[(observations['obs_code'] == '69453-9') &   # Death
                                (observations['obs_value'].str.contains('overdose'))]['obs_pat_id'])

# Determine which patients were ever prescribed opioids
patients_prescribed_opioids = set(medications[medications['med_code'].isin(opioids_rxnorm)]['med_pat_id'])  # Opioids
df['prescribed_opioids'] = np.where(df['pat_id'].isin(patients_prescribed_opioids), 1, 0)

print('Num patients prescribed opioids = {}, Num overdoses = {}, Num overdose deaths = {}'
         .format(len(patients_prescribed_opioids),len(patients_that_overdosed),len(patients_overdose_deaths)))

### 4.2.1 Compute the days_supply variable

We want to compute how many days supply of a medication a patient was prescribed at discharge.  The approach we will use is that for each encounter, we will find all of the medications associated with the encounter.  We will look for medicates that are opioids and find the largest days supply for that encounter and store the result in the 'opioid_discharge_days_supply' column.

This is most easily accomlished using a function that defines the logic and the ".apply" DataFrame function the will iterate over each row in a DataFrame, call the function and store the result back in the DataFrame.

In [ ]:
# Define the function that will perform to logic of compute the discharge opioid days supply
from random import random
# TODO Fix in CreateData
def get_days_supply(pat_id):
    enc_meds = medications[medications['med_pat_id'] == pat_id]
    enc_opioid_meds = enc_meds[enc_meds['med_code'].isin(opioids_rxnorm)]
    max = 0
    if len(enc_opioid_meds) > 0:
        try:
            max = int(enc_opioid_meds['med_days_supply'].max())
        except ValueError:
            max = 0
    return int(max)

# Apply the function to each row (Note: this can take a little while to finish)
df['opioid_discharge_days_supply'] = df.apply(lambda x: get_days_supply(x['pat_id']), axis=1)

# Display the first 5 entries that have a non-zero days supply, just to check our logic
df[df['opioid_discharge_days_supply'] > 0].head(5)

# 5.0 Explore the Data - Part 2

### 5.1 Outcome variable

Now that we have created some new variables, lets take a look at how the outcome variable is associated with our predictors.

In [ ]:
# See who overdosed from prescribed opioids by computing the intersection

overlap = patients_that_overdosed.intersection(patients_prescribed_opioids)

print('Num that overdose = {}, Num that were prescribed opioids = {}, overlap = {}'.format(\
    len(patients_that_overdosed),len(patients_prescribed_opioids),len(overlap)))


How many patients overdosed?

Since we store 'overdose' as a 0 or 1, we can just use the mean function to compute what percent of the population overdosed


In [ ]:
ov = patients[patients['pat_id'].isin(patients_prescribed_opioids)]['overdose']
display(ov.value_counts())
print('Percent that overdosed: {0:.2f}%'.format(ov.mean()*100))

In [ ]:
patients.groupby('overdose').mean()


In [ ]:
# What was the mean number of days_supply for patients that overdosed and were prescribed opioids?

ct = pd.crosstab(df['prescribed_opioids'],df['opioid_discharge_days_supply'])
ct.iloc[1][1:].plot()
#sns.factorplot(data=ct[0], x='')
df[['overdose','opioid_discharge_days_supply']].groupby('overdose').mean()

### 5.3 Graph the patient variables against the outcome

Let's see if gender, race and age are associated with the outcome

In [ ]:
pd.crosstab(patients['gender'],patients['overdose']).plot(kind='bar')
plt.title('Overdose by Gender')
plt.xlabel('Gender')
plt.ylabel('Overdose?')

In [ ]:
# Number of Male vs Female overdoses
display(pd.crosstab(patients['gender'],patients['overdose'],margins=True))
display(pd.crosstab(patients['race'],patients['overdose'],margins=True))
display(pd.crosstab(patients['race'],patients['overdose'],normalize='index'))
pd.crosstab(patients['ethnicity'],patients['overdose'],normalize='index')

In [ ]:
pd.crosstab(patients['race'],patients['overdose']).plot(kind='bar')
plt.title('Overdose by Race')
plt.xlabel('Race')
plt.ylabel('Overdose?')

We have a pretty uniform distribution of ages in the patient data

In [ ]:
patients['age'].hist()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')

In [ ]:
# Graph age_at_visit with probability of overdose
# TODO: FIX

ct = pd.crosstab(df['age'],df['overdose'],normalize='index')[1]

#ct.hist()
ct.plot()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Probability of Overdose')
plt.show()

### 5.4 Grouping by a variable

We often want to group related rows together and then count the number rows of each type or find the mean of a variable for each row type.

For example, lets count how many encounters each patient has over the timeframe of the data.  We will use the `groupby` function to group on a set of variables.  The operation returns a `groupby` object which doesn't actually group the data but instead acts like a set of instructions telling the DataFrame how to group itself.  We need to apply another function, such as size(), mean() or sum(), to the groups to yield a result.

In [ ]:
# How many encounters does each patient have?

encs = encounters.groupby(['enc_pat_id']).size()

#display(encs[0:5])
#display(type(encs))

# We can store that information directly into the patients DataFrame since `encs` is indexed by the pat_id
patients = patients.set_index('pat_id')
patients['num_encounters'] = encs
patients = patients.reset_index()

#display(patients.head(5))

sns.lmplot(data=patients,x='age',y='num_encounters',hue='gender')

## For those that overdose, what is the days_supply?

In [ ]:
display(df[df['overdose']==1].mean())
df[df['prescribed_opioids']==1].mean()

### What are the primary reasons for visit for those that overdosed?

In [ ]:
encounters[encounters['enc_pat_id'].isin(patients_that_overdosed)]['enc_reason_description'].value_counts(sort=True)

## For patients that overdosed, what was the most frequent condition?

In [ ]:


conditions[conditions['cond_pat_id'].isin(patients_that_overdosed)]['cond_description'].value_counts(sort=True)

### Get an idea of how a patient progresses through their healthcare

When exploring the data, it helps to visualize what is happening across time.  You can create small functions within the Jupyter notebook and reuse them further down in the notebook.  In this case, we are looping through the encounter data for a patient and print all of the medications and labs (observations) that are associated with the patient.  The function 'display_trajectory' is passed the id for a patient and then prints the information.  We can use this later to further examine data or debug things we don't understand.

In [ ]:
pt_id = '3eaed230-1c60-4221-a96c-f6af5d871072'
#pt = patients.query('pat_id == @pt_id')
#pt = patients[patients.pat_id == pt_id].iloc[0]
#pt = patients.loc[pt_id]

def display_trajectory(df,pt_id):
    pt = patients[patients['pat_id']==pt_id]

    display(pt)
    encs = encounters[encounters.enc_pat_id == pt_id]
    #print(encs.shape)
    for i, e in encs.iterrows():
        #dt = df[df['pat_id']==e['enc_pat_id']].iloc[0]
        print('  {:%Y-%m-%d}: {} ({}) ({})'.format(e['enc_date'], e['enc_description'], \
                         e['enc_code'], e['enc_reason_description']))
        meds = medications[medications['med_enc_id'] == e['enc_id']]
        for j, m in meds.iterrows():
            print('     MED: {:%Y-%m-%d}: {} ({}) days_supply={}'.format(m['med_start_date'],  \
                                            m['med_description'], m['med_code'], m['med_days_supply']))
        labs = observations[observations['obs_enc_id'] == e['enc_id']]
        for k, l in labs.iterrows():
            print('     LAB: {:%Y-%m-%d %H:%M}: {} ({}) {} {}'.format(l['obs_date'], l['obs_description'], l['obs_code'], l['obs_value'], l['obs_units']))
            
            

In [ ]:
display_trajectory(df,patients.iloc[0].pat_id)

In [ ]:
# Who were the patients that overdosed?

pt = df[df['pat_id'].isin(patients_that_overdosed)]

pt.head(10)
#display(pd.DataFrame([pt['first'],pt['last'],pt['marital'],pt['race'],pt['gender'],pt['ethnicity']]).T)

In [ ]:
# Display the trajectory of one of those patients
pt_id = list(patients_that_overdosed)[1]
display_trajectory(df,pt_id)

# 6.0 Modeling

We will be using the `scikit-learn` package, which is an open-source library that provides a robust set of machine learning algorithms for Python. It is built upon the core Python scientific stack and has a simple, consistent interface.

<img src="http://1.bp.blogspot.com/-ME24ePzpzIM/UQLWTwurfXI/AAAAAAAAANw/W3EETIroA80/s1600/drop_shadows_background.png" width="800px"/>



In [ ]:
# Load the modules
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
import random as rnd
from random import random, randint

Most of the models in scikit-learn require the categorical variables be turned into numeric variables.  There are two approaches to this:
1. One Hot Encoding - each item in the categorical variable is turned into its own variable represetnting the presence or abscence of that item.  For example, 'Gender' would turn into 2 variables:  'Gender_M' and 'Gender_F'
2. Label Encoding - assign an integer to each item.  For the 'Gender' variable, 0 might mean Male and 1 might mean Female.

We will use the `LabelEncoder` transformation to change categorical variables into integers.  As a convenience, we can also keep the original (human readable) variable in the Dataframe.

In [ ]:
# Let's use the following variables as our initial set of predictors
cat_cols = ['gender', 'marital', 'race', 'ethnicity']
cat_cols_encoded = [c + '_encoded' for c in cat_cols]
numeric_cols = ['prior_opioid_abuse_diag', 'age', 'opioid_discharge_days_supply']
pred_cols = numeric_cols + cat_cols_encoded
target_col = 'overdose'
all_cols = cat_cols+numeric_cols+[target_col]

df_opioids = df[df['prescribed_opioids'] == 1]
print(df_opioids.shape)

# Encode the categorical variables
dfe = df_opioids[cat_cols]
print(dfe.shape)
# Replace missing data with an 'Unknown' category so the missing data will also be encoded
dfe = dfe.replace(np.NaN,'Unknown')

# Encode the categorical variables
encoded = dfe.apply(preprocessing.LabelEncoder().fit_transform)

# Append the non-categorical variables and the encoded variables into a single Dataframe
# Name the new variables as <name>_encoded
dfe = pd.concat([df_opioids[all_cols], encoded.add_suffix('_encoded')],axis=1)

In [ ]:
display(df_opioids.corr())

In [ ]:
# df_opioids.loc[:,'prior_opioid_abuse_diag'] = df_opioids['overdose'].apply(lambda x: x if random() > .90 else 1)
# df_opioids.corr()

In [ ]:
# Lets try to build a model using this set of variables
pred_cols = ['age', 'opioid_discharge_days_supply', 
             'prior_opioid_abuse_diag', \
             'gender_encoded', 'marital_encoded', 'race_encoded', 'ethnicity_encoded']
#pred_cols = ['prior_abuse_diag', 'adult', 'age_at_visit', 'opioid_discharge_days_supply', \
#             'gender_encoded', 'marital_encoded', 'race_encoded', 'ethnicity_encoded']

LR_pred_cols = pred_cols
X = dfe[pred_cols].as_matrix()
y = dfe['overdose'].as_matrix()

To remind us of the definitions of recall (sensitivity) and precision (positive predictive value), here is a graphic from Wikipedia:

![Confusion Matrix](Sensitivity-Wikipedia.png)

To quote from Scikit Learn:

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier to not label a sample as positive if it is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

Accuracy is (tp + tn) / N.

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.

The F-beta score weights the recall more than the precision by a factor of beta. beta = 1.0 means recall and precision are equally important.

The support is the number of occurrences of each class in y_test.

In [ ]:
# Let's try a simple logistic regression model to see how predictive our data is

# https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
# 

LR = LogisticRegression()
result = LR.fit(X,y)

expected = y
predicted = LR.predict(X)

print('\nClassification Report\n',metrics.classification_report(expected, predicted))
print('\nConfusion Matrix\n',metrics.confusion_matrix(expected, predicted))
print('\nAccuracy score =',metrics.accuracy_score(expected, predicted))
print('\nAUC score =',metrics.roc_auc_score(expected, predicted))
print('\nf1 score =',metrics.f1_score(expected, predicted))



In [ ]:
# Lets graph an ROC curve for the model

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
auc = roc_auc_score(y, LR.predict(X))

probs = LR.predict_proba(X)[:,1]
fpr, tpr, thresholds = roc_curve(y, probs)
tpr[1] = tpr[0]

plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Model')
plt.legend(loc="lower right")
plt.savefig('ROC1.png')


In [ ]:
# Now lets try it with Dummy Variables (One Hot Coding)

# 7.0 Model Evaluation

We built a Logistic Regression model that was moderatly predictive of an overdose.  But we trained the model and tested it with our entire data set, which isn't exactly a fair test of the model.

We can split our dataset into a training dataset and a test dataset.  After we train the model using just the training dataset, we will evaluate the model using the test dataset which has data that the model has never seen before.

scikit-learn has a nice function called `train_test_split` that randomly splits our dataset.


In [ ]:
# Create the training and test datasets
train, test = train_test_split(dfe, test_size=0.3, random_state=987)

X_train = train[pred_cols].as_matrix()
y_train = train['overdose'].as_matrix()
X_test = test[pred_cols].as_matrix()
y_test = test['overdose'].as_matrix()

### 7.1 Logistic Regression

In [ ]:
LR = LogisticRegression()
result = LR.fit(X_train,y_train)
print(LR)
expected = y_train
predicted = LR.predict(X_train)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

expected = y_test
predicted = LR.predict(X_test)

print('\nClassification Report\n',metrics.classification_report(expected, predicted))
print('\nConfusion Matrix\n',metrics.confusion_matrix(expected, predicted))
print('\nAccuracy score =',metrics.accuracy_score(expected, predicted))
print('\nAUC score =',metrics.roc_auc_score(expected, predicted))
print('\nf1 score =',metrics.f1_score(expected, predicted))

### 7.2 Graph an ROC curve

In [ ]:
# Lets graph an ROC curve for the model

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
auc = roc_auc_score(y_test, LR.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, LR.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Model (train/test split)')
plt.legend(loc="lower right")
plt.savefig('ROC2.png')

### 7.3 Cross Validation

Splitting the data into a training dataset and a test dataset has a drawback in that it doesn't allow us to train the model on all of the data.  Another approach is to use k-fold cross validation where we randomly divide the data into k equal sized subsets and then train the model on the remaining k-1 segments of the data and test it on the data that we held out.  The process is repeated k times so that in the end all of the data is used at some point to train the model.  The k model results are averaged to produce a single estimate of model performance.

In [ ]:
kfold = model_selection.KFold(n_splits=10, random_state=387)
results = model_selection.cross_val_score(LR, X_train, y_train, cv=kfold, scoring='roc_auc')

print('Model score = %.4f (%.4f)' %(results.mean(), results.std()))


### 7.4 Testing many models

The nice thing about scikit-learn is that most of the models have similar calling signatures so we can just create a list of all of the models we are interested in testing and then invoke each model in a for loop.

We can train each model in turn and then use k-fold cross validate to estimate model performance.

In [ ]:
# Cross validation test harness 

# Make this into a function so we can reuse it later

def eval_models(X, y):
    seed = 543
    # prepare models
    models = []
    models.append(('LR', LogisticRegression(class_weight='balanced')))
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeClassifier()))
    models.append(('NB', GaussianNB()))
    #models.append(('SVM', SVC()))
    models.append(('RF', RandomForestClassifier(n_estimators=100, class_weight='balanced')))
    # evaluate each model in turn
    results = []
    names = []
    #scoring = 'accuracy'
    scoring = 'roc_auc' # others include: 'accuracy', 'f1', 'roc_auc', 
                        # or found here: http://scikit-learn.org/stable/modules/model_evaluation.html
    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    return results, names

# Execute the function
results, names = eval_models(X_train,y_train)  #TODO:  X,y

### 7.5 Graphically viewing model performance using a boxplot

None of the models do very well.  LogisticRegression does the best but most of the models have quite a bit of variance in their scores.


In [ ]:
# boxplot algorithm comparison
def plot_perf(names,results):
    fig = plt.figure()
    fig.suptitle('Algorithm Comparison')
    ax = fig.add_subplot(111)
    plt.boxplot(results)
    plt.ylabel('roc_auc')
    ax.set_xticklabels(names)
    plt.show()
    
plot_perf(names,results)

### 7.6 Model Improvements

In [ ]:
# Can we improve the model by adding some variables?
new_cols = ['prior_opioid_abuse_diag',  'age',  \
             'gender_encoded', 'marital_encoded', 'race_encoded', 'ethnicity_encoded']
print(new_cols)

X = dfe[new_cols].as_matrix()
y = dfe['overdose'].as_matrix()

X_train = train[new_cols].as_matrix()
y_train = train['overdose'].as_matrix()
X_test = test[new_cols].as_matrix()
y_test = test['overdose'].as_matrix()

results, names = eval_models(X,y)

plot_perf(names,results)

### 7.7 Assess Variable Importance

In [ ]:
# Display relative importance of each variable
pred_cols = ['gender_encoded', 'marital_encoded', 'race_encoded', 'ethnicity_encoded', 'age', 
               'prior_opioid_abuse_diag', 'opioid_discharge_days_supply']
model = RandomForestClassifier()
model.fit(dfe[pred_cols], dfe['overdose'])

# display the relative importance of each attribute
display(pd.DataFrame(list(zip(pred_cols,model.feature_importances_))))



### 7.8 Predict the risk for new patients

In [ ]:
# Show how to use the resulting model to predict opioid overdose
# age, opioid_discharge_days_supply, prior_opioid_abuse_diag, gender (F), marital (M), race (white), ethnicity (english)
new_patient = [45,10,1,0,1,4,7]

pred = LR.predict(np.asmatrix(new_patient))
if pred[0] == 0:
    print('Patient has no overdose risk.')
elif pred[0] == 1:
    print('Patient has overdose risk.')